In [49]:
# @title
import warnings
warnings.filterwarnings('ignore')


try:
    import cirq
except ImportError:
    print('installing cirq...')
    %pip install cirq --quiet
    import cirq
    print('installed cirq.')


#!git clone https://github.com/the-codingschool/bb84.git
#from bb84.bb84 import BB84

from cirq import NamedQubit, Simulator, Circuit

class BB84:

    """
    Implements the BB84 protocol for quantum key distribution.

    The BB84 protocol allows two parties, Alice and Bob, to generate a shared secret key, which can then be used
    for secure communication. The protocol also involves checking for the presence of an eavesdropper, Eve.
    This implementation provides a simulated environment for the protocol's execution, including the potential
    interception by Eve.


    <b>NOTE</b>:
        This class requires external definitions for the quantum circuits and simulator to function properly.

    <br>

    © 2024 The Coding School, All rights reserved.
    """

    qubit: NamedQubit
    """The qubit being used for the protocol. Must be defined externally, for example:
    ```python
    protocol.qubit = cirq.NamedQubit('q0')
    ```
    """

    simulator: Simulator
    """The quantum circuit simulator. Must be defined externally, for example:
    ```python
    protocol.simulator = cirq.Simulator()
    ```
    """

    alice_send_0_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_0_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_H_circuit = cirq.Circuit()
    ```
    """

    eve_intercept_circuit: Circuit
    """The quantum circuit that Eve will use to intercept the qubit between Phase 1 and 2. Must be defined externally, for example:
    ```python
    protocol.eve_intercept_circuit = cirq.Circuit()
    ```
    """

    bob_receive_no_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 without using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_no_H_circuit = cirq.Circuit()
    ```
    """

    bob_receive_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_H_circuit = cirq.Circuit()
    ```
    """


    eve_intercept: str
    """Indicates if Eve intercepts the qubits ('yes' or 'no'). Should be set upon initialization or after calling `reset()`."""

    alice_bit: int
    """The current bit Alice wants to send. Should only be modified indirectly through the `send_bit` method's parameters."""

    bob_bit: int
    """The bit received by Bob after measurement. Should only be modified indirectly through the `send_bit` method's parameters."""

    eve_bit: int
    """The bit measured by Eve if interception occurs. Should only be modified indirectly through the `send_bit` method's parameters."""

    does_alice_apply_H: str
    """Indicates if Alice applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""

    does_bob_apply_H: str
    """Indicates if Bob applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""


    alice_key: list
    """The secret key generated by Alice. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bob_key: list
    """The secret key generated by Bob. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    eve_key: list
    """The key intercepted by Eve, if any. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bit_num: int
    """Counter for the number of bits processed so far. <b>SHOULD NOT BE MODIFIED BY USER</b>."""


    def __init__(self, eve_intercept = 'no'):

      """
      Initializes the BB84 protocol simulation with default or specified settings.

      Parameters:
          `eve_intercept` (str, optional): Determines if Eve will attempt to intercept the qubits ('yes' or 'no').
                                         Defaults to 'no'.

      Example Usage:
      ```python
      protocol_without_eve = BB84()

      also_protocol_without_eve = BB84(eve_intercept = 'no')

      protocol_with_eve = BB84(eve_intercept = 'yes')
      ```
      """

      self.alice_bit = None
      self.bob_bit = None

      self.eve_bit = None
      self.eve_intercept = eve_intercept

      self.does_alice_apply_H = None
      self.does_bob_apply_H = None

      self.qubit = None
      self.simulator = None

      self.alice_send_0_no_H_circuit = None
      self.alice_send_1_no_H_circuit = None
      self.alice_send_0_H_circuit = None
      self.alice_send_1_H_circuit = None

      self.eve_intercept_circuit = None

      self.bob_receive_no_H_circuit = None
      self.bob_receive_H_circuit = None

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def phase_1_circuit(self):

      """
      Creates a circuit such that Alice encodes her bit into a qubit and applies an H gate if she's chosen to do so.
      Then she sends this qubit through her quantum channel to Bob, hoping Eve does not intercept.

      <b>Returns</b>:
          The quantum circuit for Phase 1 of BB84 (through Alice sending her qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.alice_bit == 0:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_0_no_H_circuit
        else:
          return self.alice_send_0_H_circuit

      else:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_1_no_H_circuit
        else:
          return self.alice_send_1_H_circuit


    def eve_circuit(self):

      """
      Creates a circuit for Eve's role in the protocol, which depends on if she's intercepting or not.

      <b>Returns</b>:
          The quantum circuit for Eve's interception if it occurs, otherwise an empty circuit.

      <b>INTERNAL USE ONLY</b>
      """

      if self.eve_intercept == 'yes':
        return self.eve_intercept_circuit
      else:
        return cirq.Circuit()


    def phase_2_circuit(self):

      """
      Creates a circuit such that Bob receives the qubit, applies an H gate if he's decided to do so,
      and then measures it.

      <b>Returns</b>:
          The quantum circuit for Phase 2 of BB84 (through Bob measuring the qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.does_bob_apply_H == 'no':
        return self.bob_receive_no_H_circuit
      else:
        return self.bob_receive_H_circuit


    def restart(self):

      """
      Resets the protocol such that the circuits are all the same, but they are being used with blank keys and on a new quantum channel.
      """

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def send_bit(self, alice_bit, does_alice_apply_H, does_bob_apply_H, compare_bits = 'no'):

      """
      Simulates the full BB84 protocol, potentially intercepted by Eve.

      Parameters:
        <ul>
          <li>`alice_bit` (int): The bit Alice wants to send.</li>
          <li>`does_alice_apply_H` (str): Indicates if Alice applies the Hadamard gate ('yes' or 'no').</li>
          <li>`does_bob_apply_H` (str): Indicates if Bob applies the Hadamard gate ('yes' or 'no').</li>
          <li>`compare_bit` (str, optional): Determines if Alice and Bob compare their bits directly ('yes' or 'no').
                                       Defaults to 'no'.</li>
        </ul>

      <b>NOTE</b>:
          This method simulates the entire process of sending a bit, including preparation, potential interception,
          and measurement. It also handles error checking for uninitialized objects that are required in the given case.


      Example Usage:
      ```python
      # Alice sends a 1 without an H gate and Bob receives and measures the qubit without an H gate.
      # The default is that they will not compare their bits and instead will add them to their key
      # if they both made the same choice of H gate or not.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no')


      # Same as above, except Alice and Bob will compare bits instead of adding them to their key.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')
      ```
      """

      self.alice_bit = alice_bit
      self.does_alice_apply_H = does_alice_apply_H
      self.does_bob_apply_H = does_bob_apply_H

      if self.qubit == None:
        print('Error: A qubit object must be defined first.')
        return

      elif self.simulator == None:
        print('Error: A simulator object must be defined first.')
        return


      # Prepare Alice's qubit
      alice_circuit = self.phase_1_circuit()

      if alice_circuit == None:

        applies_H = 'no H'
        if self.does_alice_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Alice\'s circuit for sending a', self.alice_bit, 'with', applies_H, 'must be defined first.')
        return


      # Eve's interception
      eve_circuit = self.eve_circuit()

      if eve_circuit == None:

        print('Error: Eve\'s interception circuit must be defined first.')
        return

      # Bob's measurement
      bob_circuit = self.phase_2_circuit()

      if bob_circuit == None:

        applies_H = 'no H'
        if self.does_bob_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Bob\'s circuit for measuring with', applies_H, 'must be defined first.')
        return

      # Combine circuits and run simulation
      bb84_circuit = alice_circuit + eve_circuit + bob_circuit
      results = self.simulator.run(bb84_circuit)
      self.bob_bit = results.measurements['q0'][0][0]

      if self.eve_intercept:
        self.eve_bit = results.measurements.get('eve', [[None]])[0][0]

      # Print results
      print('\033[43m\033[1mATTEMPTED BIT', self.bit_num, '\033[0m\033[0m')

      print('\n\033[32m\033[1mPHASE 1: SENDING\033[0m\033[0m\033[0m')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: I sent a', self.alice_bit, 'and', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')

      if self.eve_intercept == 'yes':
        print('EVE INTERCEPTS!')
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: I measured a', self.eve_bit, 'and will now send the qubit to Bob')

      print('\n\033[32m\033[1mPHASE 2: RECEIVING\033[0m\033[0m')
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H and measured a', self.bob_bit)

      print('\n\033[32m\033[1mPHASE 3: COMPARING\033[0m\033[0m')
      print('Alice and Bob are comparing choice of H\'s', 'and the bits themselves.' if compare_bits == 'yes' else 'but not the bits themselves.', '\n')
      print('\033[47m\033[1mAlice\033[0m\033[0m: I', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')
      print('\033[47m\033[1mBob\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H')

      print('')
      if compare_bits == 'yes':
        print('\033[47m\033[1mAlice\033[0m\033[0m: I sent a', self.alice_bit)
        print('\033[47m\033[1mBob\033[0m\033[0m: I measured a', self.bob_bit, '\n')

        if self.does_alice_apply_H == self.does_bob_apply_H:
          if self.alice_bit == self.bob_bit:
            print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Our bits match, so it doesn\'t seem like Eve is intercepting.')

            if self.eve_intercept == 'yes':
              print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: Mwuhaha, I\'ve gone undetected.')

          else:
            print('\033[91m\033[1mAlice and Bob: Our bits are different, so Eve must have intercepted! Let\'s start over with new keys and a new quantum channel!\033[0m\033[0m')
            self.restart()

        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      else:
        if self.does_alice_apply_H == self.does_bob_apply_H:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Great, let\'s add this bit to our keys.')
          self.alice_key += [self.alice_bit]
          self.bob_key += [self.bob_bit]

          if self.eve_intercept:
            self.eve_key += [self.eve_bit]
        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      print('')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: My key is now', self.alice_key)
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: My key is now', self.bob_key)

      if self.eve_intercept == 'yes':
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: My key is now:', self.eve_key)


      print('\nThe circuit used this round:', bb84_circuit)

      self.bit_num += 1
      print('='*75, end='\n\n')

import matplotlib.pyplot as plt

In [50]:
#Process 1: Sending

#Step 1: Alice Generates a Random Key

import random

def generate_classical_key(num_bits): 
    #Step 1: Alice generates a random secret key of classical bits
    key = [random.choice(['0','1']) for _ in range(num_bits)]
    return ''.join (key)

def alice_prepare_bb84(key, num_bits):
    #Create one qubit per bit
    qubits = [cirq.NamedQubit(f'a{i}') for i in range(num_bits)]
    circuit = cirq.Circuit()
    bases = []

    #Convert classical bits to qubit states and apply random bases
    for i, (qubit, bit) in enumerate(zip(qubits,key)):
        basis = random.choice([0,1]) #0: rectilinear(Z), 1:diagonal(X)
        bases.append(basis)

        #Convert classical bit to qubit state
        if bit == '1':
            circuit.append(cirq.X(qubit))

        #Apply H-gate for diagonal basis
        if basis == 1:
            circuit.append(cirq.H(qubit)) 
            #Applies H-gate for bit randomly chosen to be in X-basis
    return circuit, bases, qubits

#Generate classical key
num_bits= 16
secret_key = generate_classical_key(num_bits)
print(f"Alice's Classical Secret Key ({num_bits} bits): {secret_key}")

#Alice prepares quantum states
alice_circuit, alice_bases, qubits = alice_prepare_bb84(secret_key, num_bits)
print("\nAlice's Basis Choices (0=Rectilinear, 1=Diagonal):", alice_bases)

#Print the Circuit
print("\nAlice's Circuit for All Bits:")
print(alice_circuit)

#Print state preparation for each bit
print("\nState Preparation for Each Bit:")
for i, (bit, basis) in enumerate(zip(secret_key, alice_bases)):
    print(f"Bit {i} (key={bit}, basis={basis}): q{i}")

Alice's Classical Secret Key (16 bits): 0111010100010010

Alice's Basis Choices (0=Rectilinear, 1=Diagonal): [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0]

Alice's Circuit for All Bits:
a1: ────X───H───

a2: ────X───H───

a3: ────X───H───

a5: ────X───────

a6: ────H───────

a7: ────X───H───

a8: ────H───────

a11: ───X───H───

a12: ───H───────

a14: ───X───────

State Preparation for Each Bit:
Bit 0 (key=0, basis=0): q0
Bit 1 (key=1, basis=1): q1
Bit 2 (key=1, basis=1): q2
Bit 3 (key=1, basis=1): q3
Bit 4 (key=0, basis=0): q4
Bit 5 (key=1, basis=0): q5
Bit 6 (key=0, basis=1): q6
Bit 7 (key=1, basis=1): q7
Bit 8 (key=0, basis=1): q8
Bit 9 (key=0, basis=0): q9
Bit 10 (key=0, basis=0): q10
Bit 11 (key=1, basis=1): q11
Bit 12 (key=0, basis=1): q12
Bit 13 (key=0, basis=0): q13
Bit 14 (key=1, basis=0): q14
Bit 15 (key=0, basis=0): q15


In [51]:
#Process 2: Intercepting – Hannah

def eve_intercept_bb84(alice_circuit, qubits, num_bits):
    # Create Eve's probe qubits (one per Alice qubit)
    eve_probes = [cirq.NamedQubit(f'e{i}') for i in range(num_bits)]

    # Add entanglement operations to Alice's circuit
    for i in range(num_bits):
        qubit_0 = qubits[i]
        eve_probe = eve_probes[i]

        # Step 1: Eve prepares probe in |+⟩ = H|0⟩
        alice_circuit.append(cirq.H(eve_probe))

        # Step 2: Entangle with Alice's qubit using CNOT
        alice_circuit.append(cirq.CNOT(qubit_0, eve_probe))

        # Step 3: Eve stores the probe; original qubit continues to Bob

    return alice_circuit, eve_probes

# Eve intercepts and modifies Alice's circuit
alice_circuit, eve_probes = eve_intercept_bb84(alice_circuit, qubits, num_bits)

# Print modified circuit
print("\nCircuit After Eve's Interception (Process 2):")
print(alice_circuit)


Circuit After Eve's Interception (Process 2):
            ┌───────────┐   ┌─────┐
a0: ──────────@───────────────────────
              │
a1: ────X────H┼──────────────@────────
              │              │
a2: ────X────H┼──────────────┼@───────
              │              ││
a3: ────X────H┼──────────────┼┼@──────
              │              │││
a4: ──────────┼@─────────────┼┼┼──────
              ││             │││
a5: ────X─────┼┼@────────────┼┼┼──────
              │││            │││
a6: ────H─────┼┼┼@───────────┼┼┼──────
              ││││           │││
a7: ────X────H┼┼┼┼───────────┼┼┼@─────
              ││││           ││││
a8: ────H─────┼┼┼┼@──────────┼┼┼┼─────
              │││││          ││││
a9: ──────────┼┼┼┼┼@─────────┼┼┼┼─────
              ││││││         ││││
a10: ─────────┼┼┼┼┼┼@────────┼┼┼┼─────
              │││││││        ││││
a11: ───X────H┼┼┼┼┼┼┼────────┼┼┼┼@────
              │││││││        │││││
a12: ───H────@┼┼┼┼┼┼┼────────┼┼┼┼┼────
             ││││││││       

In [52]:
#Process 3: Receiving

def bob_measure_bb84(alice_circuit, qubits, num_bits):
    bob_bases = []
    bob_measurements = []

    #Creates random measurement in the Z or X basis
    for i in range(num_bits):
        qubit = qubits[i]
        bob_basis = random.choice([0, 1])  # 0: Z basis, 1: X basis
        bob_bases.append(bob_basis)

        # Create a new circuit for this qubit
        new_circuit = cirq.Circuit()

        # Includes Alice’s gates for this qubit
        for moment in alice_circuit:
            for op in moment:
                if qubit in op.qubits:
                    new_circuit.append(op)

        if bob_basis == 1:
            new_circuit.append(cirq.H(qubit))

        new_circuit.append(cirq.measure(qubit, key='m'))

        sim = cirq.Simulator()
        result = sim.run(new_circuit, repetitions=1)
        bit = result.measurements['m'][0][0]
        bob_measurements.append(bit)

    return bob_bases, bob_measurements


# Call Bob's measurement function
bob_bases, bob_measurements = bob_measure_bb84(alice_circuit, qubits, num_bits)

# Print Bob's measurement results
print("\nBob's Measurement Results:")
for i in range(num_bits):
    print(f"Bit {i} (basis={bob_bases[i]}, measurement={bob_measurements[i]}): a{i}")



Bob's Measurement Results:
Bit 0 (basis=0, measurement=0): a0
Bit 1 (basis=0, measurement=1): a1
Bit 2 (basis=1, measurement=0): a2
Bit 3 (basis=1, measurement=0): a3
Bit 4 (basis=0, measurement=0): a4
Bit 5 (basis=1, measurement=0): a5
Bit 6 (basis=1, measurement=1): a6
Bit 7 (basis=1, measurement=1): a7
Bit 8 (basis=1, measurement=1): a8
Bit 9 (basis=1, measurement=1): a9
Bit 10 (basis=0, measurement=0): a10
Bit 11 (basis=0, measurement=1): a11
Bit 12 (basis=1, measurement=0): a12
Bit 13 (basis=1, measurement=1): a13
Bit 14 (basis=1, measurement=1): a14
Bit 15 (basis=1, measurement=0): a15


In [95]:
# Process 4: Comparing (Reconciliation Stage)
def reconcile_keys(alice_bases, bob_bases, secret_key, bob_measurements, num_bits):
    sifted_key_alice = []
    sifted_key_bob = []
    matching_indices = []
    # Compare bases and keep bits where bases match
    for i in range(num_bits):
        if alice_bases[i] == bob_bases[i]:
            sifted_key_alice.append(secret_key[i])
            sifted_key_bob.append(str(bob_measurements[i]))
            matching_indices.append(i)
    # Error checking: Calculate error rate by comparing sifted keys
    errors = sum(1 for a, b in zip(sifted_key_alice, sifted_key_bob) if a != b)
    error_rate = errors / len(sifted_key_alice) if sifted_key_alice else 0
    return sifted_key_alice, sifted_key_bob, matching_indices, error_rate
sifted_key_alice, sifted_key_bob, matching_indices, error_rate = reconcile_keys(
    alice_bases, bob_bases, secret_key, bob_measurements, num_bits
)
print(f"Sifted Key (Alice): {''.join(sifted_key_alice)}")
print(f"Sifted Key (Bob): {''.join(sifted_key_bob)}")
print(f"Matching Indices: {matching_indices}")
print(f"Error Rate: {error_rate:.3f}")

Sifted Key (Alice): 011001000
Sifted Key (Bob): 000011100
Matching Indices: [0, 2, 3, 4, 6, 7, 8, 10, 12]
Error Rate: 0.444


In [ ]:
def eve_measure_probes(alice_circuit, eve_probes, matching_indices, alice_bases, num_bits):
    eve_measurements = []
    for i in matching_indices:
        eve_probe = eve_probes[i]
        basis = alice_bases[i]  # Use the basis announced by Alice/Bob
        # Create a new circuit for Eve’s measurement
        eve_circuit = cirq.Circuit()
        # Include only Eve’s probe preparation and entanglement
        for moment in alice_circuit:
            for op in moment:
                if eve_probe in op.qubits:
                    eve_circuit.append(op)
        # Measure in the same basis as Alice/Bob
        if basis == 1:  # Diagonal (X) basis
            eve_circuit.append(cirq.H(eve_probe))
        eve_circuit.append(cirq.measure(eve_probe, key=f'e{i}'))
        sim = cirq.Simulator()
        result = sim.run(eve_circuit, repetitions=1)
        bit = result.measurements[f'e{i}'][0][0]
        eve_measurements.append(bit)
        eve_measurements = [int(x) for x in eve_measurements]
    return eve_measurements

eve_measurements = eve_measure_probes(alice_circuit, eve_probes, matching_indices, alice_bases, num_bits)
print(f"Eve’s Measurements for Matching Indices: {eve_measurements}")
# Compare Eve’s measurements with Alice’s sifted key
correct_guesses = sum(1 for e, a in zip(eve_measurements, sifted_key_alice) if str(e) == a)
info_gain = correct_guesses / len(sifted_key_alice) if sifted_key_alice else 0
print(f"Eve’s Information Gain (fraction of correct bits): {info_gain:.3f}")

Eve’s Measurements for Matching Indices: [1, 0, 0, 1, 0, 0, 0, 1, 0]
Eve’s Information Gain (fraction of correct bits): 0.333
